## Modules

In [25]:
import pandas as pd
import Levenshtein
import string, re
from torchmetrics.text.rouge import ROUGEScore

## Config

In [26]:
input_csv =  "results/test_llama_13b_finetuned_not_all_answerable_4bit.csv"
output_csv = input_csv.split('.')[0] + "_EXPANDED.csv"

## Evaluation

In [27]:
result_df = pd.read_csv(input_csv, sep=';', index_col=0)
result_df

,reference,question,correct_answer,answer_w_ref,answer_no_ref,rouge1_fmeasure
0,1943 novemberében Hitler Erwin Rommelt nevezte...,Miért Rommelt nevezte ki Hitler a francia védm...,Nincs elegendő adat a kérdés megválaszolásához.,Nincs elegendő adat a kérdés megválaszolásához...,"Nem létezik megbízható forrás, amely megerősít...",0.956522
1,A gyermeket hét-nyolc éves korában az 1692–169...,Mikor veszítette el édesanyját Johann Sebastia...,A korabeli feljegyzések szerint a gyermek Bach...,"A fiú korán, mindössze kilencévesen veszítette...","Johann Sebastian Bach édesanyja, Maria Elisabe...",0.575758
2,A telepes lakásépítés a századfordulón egyre n...,Hány lakóháazt építettek a Wekerle-telepre?,A kor legismertebb példája a budapesti Wekerle...,A telepes lakásépítés a századfordulón egyre n...,Nincs adat a Wekerle-telep lakóházainak számáról.,0.553191
3,a szennyvíz és a szennyezett levegő tisztításá...,Kitől kell függetlennek lennie az OEKO-TEX STe...,Nincs elegendő adat a kérdés megválaszolásához.,"Az üzemet egy arra illetékes, az OEKO-TEX Szer...",Az OEKO-TEX STeP auditot egy olyan személy vég...,0.064516
4,1833. október 13-án Ilja Petrovics újból megnő...,Ilja Petrovics Csajkovszkijnak és Alekszandra...,"Házasságukból hét gyermek született, de első l...","Házasságukból hét gyermek született, de első l...",Ilja Petrovics Csajkovszkij és Alekszandra And...,0.967742
...,...,...,...,...,...,...
678,A Maine-öbölben a szervezett bálnafigyelésekko...,Milyen bálnák láthatók a Maine-öbölben?,Nincs elegendő adat a kérdés megválaszolásához.,A Maine-öbölben a szervezett bálnafigyelésekko...,A Maine-öbölben a következő bálnafajok láthatók:,0.206897
679,Az Intelmek Szent István király intelmei Imre ...,Mit jelent az admonitiones szó?,"A király fiához, a trónörökös Imre herceghez i...",Nincs elegendő adat a kérdés megválaszolásához...,Az admonitiones (azaz figyelmeztetések) a kora...,0.228571
680,Magyar Királyi Állami Főgimnázium (Tündérpalot...,Melyik útról nyílik a volt Magyar Királyi Álla...,Magyar Királyi Állami Főgimnázium (Tündérpalot...,Magyar Királyi Állami Főgimnázium (Tündérpalot...,Melyik útról nyílik a volt Magyar Királyi Álla...,0.887460
681,"1981-ben adódott az első lehetőség arra, hogy ...",Mikor kapta meg Puskás a Kispest díszpolgára c...,1991-ben végleg Magyarországra költözött. Még ...,1991-ben végleg Magyarországra költözött. Még ...,A Kispesti Munkás Testedző Kör (KMTE) 1967. má...,0.977778


In [28]:
def remove_articles(text):
    regex = re.compile(r"\b(a|an|the)\b", re.UNICODE)
    return re.sub(regex, " ", text)

def white_space_fix(text):
    return " ".join(text.split())

def remove_punc(text):
    exclude = set(string.punctuation)
    return "".join(ch for ch in text if ch not in exclude)

def lower(text):
    return text.lower()


def normalize_text(s):
    """Removing articles and punctuation, and standardizing whitespace are all typical text processing steps."""

    return white_space_fix(remove_articles(remove_punc(lower(s))))

def compute_exact_match(prediction, truth):
    return int(normalize_text(prediction) == normalize_text(truth))


def compute_f1(prediction, truth):
    pred_tokens = normalize_text(prediction).split()
    truth_tokens = normalize_text(truth).split()
    
    # if either the prediction or the truth is no-answer then f1 = 1 if they agree, 0 otherwise
    if len(pred_tokens) == 0 or len(truth_tokens) == 0:
        return int(pred_tokens == truth_tokens)
    
    common_tokens = set(pred_tokens) & set(truth_tokens)
    
    # if there are no common tokens then f1 = 0
    if len(common_tokens) == 0:
        return 0
    
    prec = len(common_tokens) / len(pred_tokens)
    rec = len(common_tokens) / len(truth_tokens)
    
    return 2 * (prec * rec) / (prec + rec)

In [29]:
def find_most_similar_substring_levenshtein(string1, string2) -> (str, int):
    best_match = ""
    min_distance = float('inf')

    for i in range(len(string1) - len(string2) + 1):
        substring = string1[i:i+len(string2)]
        distance = Levenshtein.distance(substring, string2)

        if distance < min_distance:
            min_distance = distance
            best_match = substring

    return best_match, min_distance

def relative_levenshtein_distance(prediction, truth):
    return Levenshtein.distance(prediction, truth) / len(truth)

In [30]:
failed_extraction_count = 0
all_extraction_count = 0

f1_col = []
f1_all = 0

rel_levenshtein_col = []

rouge = ROUGEScore()
rouge_col = []
r_scores_sum=0

for index, row in result_df.iloc[:].iterrows():
    given_answer = row["answer_w_ref"].replace("</s>", '')
    
    f1_score = compute_f1(given_answer, row["correct_answer"])
    f1_col.append(f1_score)
    f1_all = f1_all + f1_score
    
    relative_levenshtein = relative_levenshtein_distance(given_answer, row["correct_answer"])
    rel_levenshtein_col.append(relative_levenshtein)
    
    r_score = float(rouge(given_answer, row["correct_answer"])['rouge1_fmeasure'])
    r_scores_sum = r_scores_sum + r_score
    rouge_col.append(r_score)
    
    if given_answer != "Nincs elegendő adat a kérdés megválaszolásához.":
        all_extraction_count = all_extraction_count + 1
        
        if given_answer not in row["reference"]:
            failed_extraction_count = failed_extraction_count + 1
            most_similar, distance = find_most_similar_substring_levenshtein(row["reference"], given_answer)
            print("Row: "+ str(index))
            print("[Question]\n" + '- ' + row["question"] + "\n")
            print("[Reference]\n" + row["reference"] + "\n")
            print("[Given answer]" + "[F1 score: " + str(f1_score) +"]\n" + "[Rouge score: " + str(r_score) +"]\n" + "\n")
            print("[Most similar part in reference]\n" + "[Levenshtein distance: "+ str(distance) +"]\n" + most_similar + '\n')
            print("[Correct answer]\n" + row["correct_answer"]) 
            print("-" * 100 + "\n\n")
        

result_df["f1_score"] =  f1_col
result_df["rel_levenshtein"] =  rel_levenshtein_col
result_df["rouge1_fmeasure"] = rouge_col
        

Row: 13
[Question]
- Az 5 évesnél fiatalabb gyerekek halálának hány százalékát okozta tüdőgyulladás?

[Reference]
2008-ban megközelítőleg 156 millió gyermek betegedett meg tüdőgyulladásban (151 millióan a fejlődő világban és 5 millióan a fejlett világban). Ez 1,6 millió halálesethez vezetett, azaz 28–34%-át okozta az ötéves kornál fiatalabb gyermekek halálozásának, ebből 95% a fejlődő világban következett be. A betegség által leginkább érintett országok közé tartoznak: India (43 millió), Kína (21 millió) és Pakisztán (10 millió). A tüdőgyulladás a vezető halálok a gyermekeknél az alacsony jövedelmű országokban. Az esetek jelentős része az újszülöttkorra esik. Az Egészségügyi Világszervezet (World Health Organization – WHO) becslése szerint minden harmadik újszülött tüdőgyulladásban hal meg.
Az esetek fele elvileg megelőzhető volna, mivel azokat olyan baktériumok okozzák, amelyek ellen már hatásos védőoltások (vakcinák) állnak rendelkezésre.


[Given answer][F1 score: 0.0579710144927536

In [31]:
print("All count: " + str(len(result_df)))
print("All extraction count: " + str(all_extraction_count))
print("Failed extraction count: " + str(failed_extraction_count))
print("Failed extraction percentage: " + str((failed_extraction_count/(all_extraction_count+failed_extraction_count)) * 100) +' %')
print("F1 avg: " + str(f1_all / len(result_df)))
print("ROUGEScore avg: "+str(r_scores_sum/len(result_df)))

All count: 683
All extraction count: 418
Failed extraction count: 49
Failed extraction percentage: 10.492505353319057 %
F1 avg: 0.6090574435891015
ROUGEScore avg: 0.6566353274092056


In [32]:
result_df.head()

,reference,question,correct_answer,answer_w_ref,answer_no_ref,rouge1_fmeasure,f1_score,rel_levenshtein
0,1943 novemberében Hitler Erwin Rommelt nevezte...,Miért Rommelt nevezte ki Hitler a francia védm...,Nincs elegendő adat a kérdés megválaszolásához.,Nincs elegendő adat a kérdés megválaszolásához...,"Nem létezik megbízható forrás, amely megerősít...",1.000000,1.000000,0.000000
1,A gyermeket hét-nyolc éves korában az 1692–169...,Mikor veszítette el édesanyját Johann Sebastia...,A korabeli feljegyzések szerint a gyermek Bach...,"A fiú korán, mindössze kilencévesen veszítette...","Johann Sebastian Bach édesanyja, Maria Elisabe...",0.553846,0.476190,0.691700
2,A telepes lakásépítés a századfordulón egyre n...,Hány lakóháazt építettek a Wekerle-telepre?,A kor legismertebb példája a budapesti Wekerle...,A telepes lakásépítés a századfordulón egyre n...,Nincs adat a Wekerle-telep lakóházainak számáról.,0.545455,0.408602,0.782528
3,a szennyvíz és a szennyezett levegő tisztításá...,Kitől kell függetlennek lennie az OEKO-TEX STe...,Nincs elegendő adat a kérdés megválaszolásához.,"Az üzemet egy arra illetékes, az OEKO-TEX Szer...",Az OEKO-TEX STeP auditot egy olyan személy vég...,0.000000,0.000000,1.829787
4,1833. október 13-án Ilja Petrovics újból megnő...,Ilja Petrovics Csajkovszkijnak és Alekszandra...,"Házasságukból hét gyermek született, de első l...","Házasságukból hét gyermek született, de első l...",Ilja Petrovics Csajkovszkij és Alekszandra And...,1.000000,1.000000,0.000000


In [33]:
result_df.to_csv(output_csv, index=True, sep=';')